#### Adds in stellar radii for systems whose stellar radii are missing in the exoarchive catalog but not in Gaia data release 2. Also, the code classifies any system whose exoplanets were detected by more than one method as 'Mixed'.

In [2]:
import csv
import numpy as np

from astropy import constants as apc

MJUPITER = apc.M_jup.value
MEARTH = apc.M_earth.value
RJUPITER = apc.R_jup.value
REARTH = apc.R_earth.value

In [3]:
class System:
    def __init__(self, sysid=None):
        self.sysid = sysid

In [4]:
#reading in file
my_file = 'C:/Users/djhoo/Documents/Oviraptor-master/catalogues/oviraptor_crossmatch_catalog.csv'  
#my_file = "/Users/research/projects/oviraptor/catalogues/oviraptor_crossmatch_catalog.csv"

raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[0]
VALUES = raw_data[1:]

def get_key(keyname, keys = KEYS, values = VALUES):
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return np.array(arr)

d = {}
d['starname'] = get_key('hostname')

d['rp'] = get_key('pl_rade')
d['rp_err1'] = get_key('pl_radeerr1')
d['rp_err2'] = get_key('pl_radeerr2')
d['mp'] = get_key('pl_bmasse')
d['mp_err1'] = get_key('pl_bmasseerr1')
d['mp_err2'] = get_key('pl_bmasseerr2')

d['ror'] = get_key('pl_ratror')
d['ror_err1'] = get_key('pl_ratrorerr1')
d['ror_err2'] = get_key('pl_ratrorerr2')

d['rs'] = get_key('st_rad')
d['rs_err1'] = get_key('st_raderr1')
d['rs_err2'] = get_key('st_raderr2')
d['ms'] = get_key('st_mass')
d['ms_err1'] = get_key('st_masserr1')
d['ms_err2'] = get_key('st_masserr2')

d['p'] = get_key('pl_orbper')
d['disc_method'] = get_key('discoverymethod')
d['temp'] = get_key('st_teff')
d['mag'] = get_key('sy_vmag')
d['ref'] = get_key('st_refname')
d['instrument'] = get_key('disc_facility')

print('Number of planets in crossmatched catalog: %d' %len(d['starname']))

Number of planets in crossmatched catalog: 4107


In [5]:
def check_uniformity(detection_methods):
    default_disc = detection_methods[0]
    status = True
    for i in range(len(detection_methods)):
        if detection_methods[i] != default_disc:
            status = False
    return status

d['system_disc_method'] = []

for i in range(len(d['starname'])):
    if check_uniformity(d['disc_method'][d['starname']==d['starname'][i]]) == False:
        d['system_disc_method'].append('Mixed')
    else:
        d['system_disc_method'].append(d['disc_method'][i])

In [ ]:
gaia_stars = {}
gaia_stars['stars'] = ['HD 136352', 'HD 141399', 'HD 160691', 'HD 20781', 'HD 20794', 'HD 27984', 'HD 31527', \
                      'HD 40307', 'tau Cet']
gaia_stars['radii'] = [1.0041078, 1.3874089, 1.3901234, 0.8336087, 0.83483344, 0.89176697, 1.0736936, 1.3310363, \
                      0.793]  

#all of these radii are taken from the Gaia archive
#except for tau Cet, which was taken from Dietrich and Apai (2020)

for i in range(len(d['starname'])):
    if d['starname'][i] in gaia_stars['stars']:
        d['rs'][i] = gaia_stars['radii'][gaia_stars['stars'] == d['starname'][i]]

In [11]:
#manually add in exoplanet radii and radii errors from HIP 41378
#This comes from Berardo et. al. (2019)
#link: https://ui.adsabs.harvard.edu/abs/2019AJ....157..185B/abstract
#all values were calculated manually starting from the radii of the star, rp/rs, and the uncertainties
#note that the order of the planets in the oviraptor catalog is c, d, e, b, f

hip = {}
hip['planet_radius'] = [2.936, 3.758, 5.411, 2.705, 9.691]   
hip['rerr_lower'] = [0.55, 0.14, 0.17, 0.084, 0.23] 
hip['rerr_upper'] = [2.6, 0.12, 0.21, 0.17, 0.23]

d['rp'][d['starname'] == 'HIP 41378'] = hip['planet_radius']
d['rerr1'][d['starname'] == 'HIP 41378'] = hip['rerr_upper']
d['rerr2'][d['starname'] == 'HIP 41378'] = hip['rerr_lower']

In [12]:
#https://arxiv.org/ftp/astro-ph/papers/0703/0703024.pdf

d['rs'][d['starname'] == 'HD 69830'] = [0.89]*len(d['starname'][d['starname'] == 'HD 69830'])
d['rserr1'][d['starname'] == 'HD 69830'] = [27.480]*len(d['starname'][d['starname'] == 'HD 69830'])
d['rserr2'][d['starname'] == 'HD 69830'] = [27.480]*len(d['starname'][d['starname'] == 'HD 69830'])

#https://arxiv.org/pdf/1402.6534.pdf

d['ms'][d['starname'] == 'Kepler-257'] = [0.9008]*len(d['starname'][d['starname'] == 'Kepler-257'])
d['ms'][d['starname'] == 'Kepler-359'] = [1.098]*len(d['starname'][d['starname'] == 'Kepler-359'])

d['mserr1'][d['starname'] == 'Kepler-257'] = [1.473]*len(d['starname'][d['starname'] == 'Kepler-257'])
d['mserr2'][d['starname'] == 'Kepler-257'] = [1.473]*len(d['starname'][d['starname'] == 'Kepler-257'])

d['mserr1'][d['starname'] == 'Kepler-359'] = [1.654]*len(d['starname'][d['starname'] == 'Kepler-359'])
d['mserr2'][d['starname'] == 'Kepler-359'] = [1.654]*len(d['starname'][d['starname'] == 'Kepler-359'])

In [13]:
npl = np.array(get_key('sy_pnum'), dtype="int")
Rstar = np.array(d['rs'])
Mstar = np.array(d['ms'])
sdm = np.array(d['system_disc_method'])
starname = np.array(d['starname'])

print("\n\nMissing RADIUS")
print(np.unique(starname[(Rstar == '')*(npl > 2)]))


print("\n\nMissing MASS")
print(np.unique(starname[(Mstar == '')*(npl > 2)]))



Missing RADIUS
['GJ 163' 'GJ 180' 'GJ 433' 'GJ 667 C' 'HD 133131 A' 'HD 133131 B'
 'HD 27894' 'HD 37124']


Missing MASS
['GJ 163']


In [14]:
WRITENEW = True
if WRITENEW:
    filepath = "/Users/research/projects/oviraptor/catalogues/oviraptor_master.csv"

    with open(filepath, "w") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(d.keys())
        writer.writerows(zip(*d.values()))